# Style transfer

We have seen that CNN's are some of the most powerful networks for image classification and analysis. CNN's process visual information in a feed forward manner, passing an input image through a collection of image filters which extract certain features from the input image. It turns out that these feature level representations are not only useful for classification, but for image construction as well. These representations are the basis for applications like Style Transfer and Deep Dream, which compose images based on CNN layer activations and extracted features. 

In this module we will focus on learning about and implementing the style transfer algorithm. Style transfer allows us to apply the style of one image to another image of our choice. The key to this technique is using  atrained CNN to separate the content from the style of an image. If we can do this then we can merge the content of one image with the style of another and create sometnhing entirely different. 

Let's talk about how style and content can be separated and by the end of this notebook, we will have all the knowledge we need to generate a stylized image of our own design. 